In [1]:
# # Load Spacy parser
# spacy.load('en')
# parser = English()

# # Function to tokenize text
# def tokenize(text):
#     lda_tokens = []
#     tokens  = parser(text)
#     for token in tokens:
#         if token.orth_.isspace():
#             continue
#         elif token.like_url:
#             lda_tokens.append('URL')
#         elif token.orth_.startswith('@'):
#             lda_tokens.append('SCREEN_NAME')
#         else:
#             lda_tokens.append(token.lower_)
#     return lda_tokens


In [2]:
# # Mallet LDA

# mallet_path = '/Users/ayankarim/mallet-2.0.8/bin/mallet'

# ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=3, id2word=id2word, iterations=1000, random_seed=100 )

In [3]:
# # Show topics
# pprint(ldamallet.show_topics(num_words=15, formatted=False))

# # Compute Coherence Score
# coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_ldamallet = coherence_model_ldamallet.get_coherence()
# print('\nCoherence Score: ', coherence_ldamallet)

In [1]:
# # Convert documents to sentences
# def sent_to_words(sentences):
#     for sentence in sentences:
#         yield(gensim.utils.simple_preprocess(str(sentence), deacc = True))
        
# data = list(df['text'])
# data_words = list(sent_to_words(data))

In [2]:
# # Create Bigrams and Trigrams

# # Build the models
# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
# trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# # fast way to get a sentece clubbed as a bigram/trigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# trigram_mod = gensim.models.phrases.Phraser(trigram)

In [3]:
# # Remove Stopwords, make bigrams and lemmatize
# stop_words = stopwords.words('english')
# stop_words.extend(['pixel', 'iphone', 'samsung', 'apple', 'essential', 'xs', 'max', 
#                   'huawei', 'galaxy', 'note', 'moto', 'oneplus', 'android', 'mate', 'pro', 'lg', 'sony', 'razer', 'phone', 'company', 
#                   'smartphone', 'google', 'thinq', 'nokia', 'htc', 'xperia', 'xz'])

# # Define functions for stopwords, bigrams, trigrams and lemmatization
# def remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]

# def make_trigrams(texts):
#     return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

In [4]:
# # Remove Stop Words
# data_words_nostops = remove_stopwords(data_words)

# # Form Bigrams
# data_words_bigrams = make_bigrams(data_words_nostops)

# # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# # python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# # Do lemmatization keeping only noun, adj, vb, adv
# data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# print(data_lemmatized[:1])

In [5]:
# # Create Dictionary
# id2word = corpora.Dictionary(data_lemmatized)

# # Create Corpus
# texts = data_lemmatized

# # Term Document Frequency
# corpus = [id2word.doc2bow(text) for text in texts]

In [6]:
# # gather tfidf scores
# tfidf = models.TfidfModel(corpus, id2word = id2word)

# # filter low value words
# low_value = 0.025

# for i in range(0, len(corpus)):
#     bow = corpus[i]
#     low_value_words = [] #reinitialize to be safe. You can skip this.
#     low_value_words = [id for id, value in tfidf[bow] if value < low_value]
#     new_bow = [b for b in bow if b[0] not in low_value_words]

#     #reassign        
#     corpus[i] = new_bow

In [7]:
# # Assigns the topics to the documents in corpus
# lda_corpus = lda[corpus]

# # Find the threshold, let's set the threshold to be 1/#clusters,
# # To prove that the threshold is sane, we average the sum of all probabilities:
# scores = list(chain(*[[score for topic_id,score in topic] \
#                       for topic in [doc for doc in lda_corpus]]))
# threshold = sum(scores)/len(scores)
# print (threshold)

# reliability = [j for i,j in zip(lda_corpus,data) if i[0][1] > threshold]
# functionality = [j for i,j in zip(lda_corpus,data) if i[1][1] > threshold]
# design = [j for i,j in zip(lda_corpus,data) if i[2][1] > threshold]

In [8]:
# labelled_df = df

# # Assign labels
# labelled_df['reliability'] = ""
# labelled_df['functionality'] = ""
# labelled_df['design'] = ""

# labelled_df['reliability'] = np.where(labelled_df['text'].isin(reliability), 'reliability', None)
# labelled_df['functionality'] = np.where(labelled_df['text'].isin(functionality), 'functionality', None)
# labelled_df['design'] = np.where(labelled_df['text'].isin(design), 'design', None)

# labelled_df['labelled'] = labelled_df[['reliability', 'functionality', 'design']].values.tolist()
# labelled_df['labelled'] = labelled_df['labelled'].apply(lambda x: list(filter(lambda a: a != None, x)))
# labelled_df = labelled_df.drop(['reliability', 'functionality', 'design'], axis=1)

In [9]:
# labelled_df.head()

In [10]:
# # Evaluate SVM Multi-Label Classifier

# y = mlb.fit_transform(labelled_df.labelled)
# X = labelled_df.text

# text_clf_svm = Pipeline([('vect', CountVectorizer()),
#                          ('tfidf', TfidfTransformer()),
#                          ('clf-svm', LabelPowerset(
#                              SGDClassifier(loss='hinge', penalty='l2',
#                                            alpha=1e-3, max_iter=6, random_state=42)))])
# _ = text_clf_svm.fit(X, y)
# predicted_svm = text_clf_svm.predict(X)

# #Calculate accuracy
# np.mean(predicted_svm == y)

In [11]:
# from sklearn import metrics
# from sklearn.metrics import confusion_matrix

# # View accuracy scores on classifying each author (precission, recall, f1-score and support)
# print(metrics.classification_report(y, predicted_svm))

In [12]:
# IPhoneX_pos = df_iphonex['pos'].sum()
# S9_pos = df_s9['pos'].sum()
# IPhoneX_neg = df_iphonex['neg'].sum()
# S9_neg = df_s9['neg'].sum()

# compare_bothphones = pd.DataFrame()
# compare_bothphones['IPhoneX_pos'] = [IPhoneX_pos]
# compare_bothphones['S9_pos'] = [S9_pos]
# compare_bothphones['pos/pos'] = [(IPhoneX_pos/S9_pos)]

# compare_bothphones['IPhoneX_neg'] = [IPhoneX_neg]
# compare_bothphones['S9_neg'] = [S9_neg]
# compare_bothphones['pos/neg'] = [(IPhoneX_neg/S9_neg)]